In [1]:
import os

subscription_id = os.getenv("<Enter Azure Subscription #>", default="<Enter Azure Subscription #>")
resource_group = os.getenv("gestaltml", default="gestaltml")
workspace_name = os.getenv("fastai2", default="fastai2")
workspace_region = os.getenv("eastus", default="eastus")

score_script = 'score_local.py'
model_name = 'breastcancerfastai_local03'
service_name = 'breastcancerlocal03'

## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [3]:
from azureml.core import Workspace

ws = Workspace.get(name=workspace_name, subscription_id=subscription_id, resource_group=resource_group)

print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

fastai2
gestaltml
eastus
40bffbcc-578f-4e44-bd6d-972552eb6513


## Register Model

In [5]:
from azureml.core.model import Model

model = Model.register(model_path="export.pkl",
                       model_name=model_name,
                       tags={'area': "breast", 'type': "transfer-learning", 'classes':'Positive,Negative'},
                       description="Breast Cancer Diag-embeded X,Y functions:092220",
                       workspace=ws)

Registering model breastcancerfastai_local03


## Open Existing Model<br>
Use if model already installed in Azure ML Workspace

In [5]:
# from azureml.core import Model
# from azureml.core.resource_configuration import ResourceConfiguration

# model = Model(ws, model_name)

# print('Name:', model.name)
# print('Version:', model.version)

Name: breastcancerfastai_local03
Version: 1


Manage Docker dependencies

In [6]:
import os

source_directory = "source_directory"

os.makedirs(source_directory, exist_ok=True)
os.makedirs(os.path.join(source_directory, "x/y"), exist_ok=True)
os.makedirs(os.path.join(source_directory, "env"), exist_ok=True)
os.makedirs(os.path.join(source_directory, "dockerstep"), exist_ok=True)

Display scoring file (var score_script)

In [8]:
with open(score_script) as f:
    print(f.read())


import os, base64
import glob
import re
import torch
import json
from io import BytesIO
from azureml.core.model import Model
from azureml.core import Workspace
import fastai 
from fastai.vision.all import *
from fastai.callback.all import *
import urllib.request



def init():
    global learner
    pathsrc=Path()

    model_path=os.getenv('AZUREML_MODEL_DIR')  
    
    filename="export.pkl"
    classes = ['Positive','Negative']
    model_path_file = os.path.join(os.environ['AZUREML_MODEL_DIR'], filename)
    model_path_hc = 'azureml-models/breastcancerfastai/1'
    print('AZUREML_MODEL_DIR-Model_Path: ' + model_path)
    print('Model_Path_File: ' + model_path_file)
    print('FastAI Version: ' + fastai.__version__)
    print ('Path(): ' + str(pathsrc))

    for filename in os.listdir(pathsrc):
        print(filename)

    for dirname, dirnames, filenames in os.walk(pathsrc):
        # print path to all subdirectories first.
        for subdirname in dirnames:
            print(os.pat

In [ ]:
# Sample for writing test data to Docker build directory
# %%writefile source_directory/extradata.json
# {
#     "people": [
#         {
#             "website": "microsoft.com", 
#             "from": "Seattle", 
#             "name": "Mrudula"
#         }
#     ]
# }

## Create Inference Configuration

 - file_path: input parameter to Environment constructor. Manages conda and python package dependencies.
 - env.docker.base_dockerfile: any extra steps you want to inject into docker file
 - source_directory: holds source path as string, this entire folder gets added in image so its really easy to access any files within this folder or subfolder
 - entry_script: contains logic specific to initializing your model and running predictions

In [9]:
#Create environment

from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.conda_dependencies import CondaDependencies

entryscript_fullpath = "x/y/" + str(score_script)

myenv = Environment(model_name)
myenv.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults~=1.13.0', 'fastcore==1.0.0', 'fastprogress==1.0.0', 'fastscript==1.0.0', 'Pillow==5.4.1', 'requests', 'torch==1.6.0', 'torchvision>=0.5.0', 'fastai==2.0.6', 'ipython'
])
# explicitly set base_image to None when setting base_dockerfile
myenv.docker.base_image = None
myenv.docker.base_dockerfile = "FROM mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04\nRUN echo \"this is test\""
myenv.inferencing_stack_version = "latest"

In [11]:
#Create inference config
inference_config = InferenceConfig(source_directory=source_directory,                                 entry_script=entryscript_fullpath,
    environment=myenv)

## Deploy Model as a Local Docker Web Service

*Make sure you have Docker installed and running.*

Note that the service creation can take few minutes.

NOTE:

The Docker image runs as a Linux container. If you are running Docker for Windows, you need to ensure the Linux Engine is running:

    # PowerShell command to switch to Linux engine
    & 'C:\Program Files\Docker\Docker\DockerCli.exe' -SwitchLinuxEngine

In [10]:
from azureml.core.webservice import LocalWebservice

# This is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

local_service.wait_for_deployment()

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 84659b36a99e4a16a8757ae52b4288dc.azurecr.io
Logging into Docker registry 84659b36a99e4a16a8757ae52b4288dc.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 84659b36a99e4a16a8757ae52b4288dc.azurecr.io/azureml/azureml_e5a6bc09d9ffd8bce12ea2b9ad46d390
 ---> 7a313ce8cfc0
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> fc3ddb08d5a4
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjQwYmZmYmNjLTU3OGYtNGU0NC1iZDZkLTk3MjU1MmViNjUxMyIsInJlc291cmNlR3JvdXBOYW1lIjoiZ2VzdGFsdG1sIiwiYWNjb3VudE5hbWUiOiJmYXN0YWkyIiwid29ya3NwYWNlSWQiOiI4NDY1OWIzNi1hOTllLTRhMTYtYTg3NS03YWU1MmI0Mjg4ZGMifSwibW9kZWxzIjp7fSwibW9kZWxzSW5mbyI6e319 | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 77f1e98ee2d7
 ---> defd71d656a8
Step 4/5 : RUN mv '/var/azureml-app/tmp5kei6ymo.py' /var/azureml-app/main.py
 ---> Running in f2089cbc0159
 ---> a26c

In [11]:
print('Local service port: {}'.format(local_service.port))

Local service port: 6789


## Check Status and Get Container Logs


In [12]:
print(local_service.get_logs())

2020-09-22T20:37:37,674009100+00:00 - rsyslog/run 
2020-09-22T20:37:37,673979200+00:00 - iot-server/run 
2020-09-22T20:37:37,673965600+00:00 - gunicorn/run 
2020-09-22T20:37:37,676363400+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_68b223042de9a84a2da2ad7149ac35f9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_68b223042de9a84a2da2ad7149ac35f9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_68b223042de9a84a2da2ad7149ac35f9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_68b223042de9a84a2da2ad7149ac35f9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_68b223042de9a84a2da2ad7149ac35f9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

## Test Web Service

Call the web service with some input data to get a prediction.

In [13]:
import json

sample_input = 'Testing Data'

print(local_service.run(sample_input))

Test,Positive


## Reload Service

You can update your score.py file and then call `reload()` to quickly restart the service. This will only reload your execution script and dependency files, it will not rebuild the underlying Docker image. As a result, `reload()` is fast, but if you do need to rebuild the image -- to add a new Conda or pip package, for instance -- you will have to call `update()`, instead (see below).

In [ ]:
%%writefile source_directory/x/y/score_template.py
import joblib
import json
import numpy as np
import os

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'sklearn_regression_local.pkl')
    # Deserialize the model file back into a sklearn model.
    model = joblib.load(model_path)

    global name, from_location
    # Note here, the entire source directory from inference config gets added into image.
    # Below is an example of how you can use any extra files in image.
    with open('source_directory/extradata.json') as json_file:  
        data = json.load(json_file)
        name = data["people"][0]["name"]
        from_location = data["people"][0]["from"]

input_sample = np.array([[10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0]])
output_sample = np.array([3726.995])

@input_schema('data', NumpyParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        result = model.predict(data)
        # You can return any JSON-serializable object.
        return "Hello " + name + " from " + from_location + " here is your result = " + str(result)
    except Exception as e:
        error = str(e)
        return error

In [63]:
local_service.reload()
print("--------------------------------------------------------------")


Container has been successfully cleaned up.
Starting Docker container...
Docker container running.
--------------------------------------------------------------


In [64]:
print(local_service.get_logs())

2020-09-23T05:58:40,990570800+00:00 - gunicorn/run 
2020-09-23T05:58:40,990989000+00:00 - iot-server/run 
2020-09-23T05:58:40,992153200+00:00 - rsyslog/run 
2020-09-23T05:58:40,995234300+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_68b223042de9a84a2da2ad7149ac35f9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_68b223042de9a84a2da2ad7149ac35f9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_68b223042de9a84a2da2ad7149ac35f9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_68b223042de9a84a2da2ad7149ac35f9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_68b223042de9a84a2da2ad7149ac35f9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

In [ ]:
# After calling reload(), run() will return the updated message.
local_service.run('Testing')


## Update Service

If you want to change your model(s), Conda dependencies, or deployment configuration, call `update()` to rebuild the Docker image.

```python

local_service.update(models=[SomeOtherModelObject],
                     deployment_config=local_config,
                     inference_config=inference_config)
```

In [ ]:
local_service.update(inference_config=inference_config)

## Delete Service

In [65]:
local_service.delete()

Container (name:distracted_hertz, id:8ad28d69da5e707bf0e44b12db78846c6aeb8f125de6fbebbb986410e78112e4) cannot be killed.
Container has been successfully cleaned up.


In [66]:
print(local_service.scoring_uri)

http://localhost:6789/score
